# Train

In [1]:
import torch
import torchvision

In [2]:
torch.cuda.empty_cache()
#device = "cpu"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Device: " + device)
print(f"Devices count: {torch.cuda.device_count()}")

Device: cuda:0
Devices count: 4


In [3]:
import pandas
import numpy
import pickle

In [4]:
from pathlib import Path

In [5]:
from misc.data import DetectorDataset, detection_results_to_annotations

In [6]:
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife import utils as pw_utils

## Data

In [7]:
data_path = Path("./data/train_data_minprirodi/")
images_path = data_path / "images"
annotation_path = data_path / "annotation.csv"

In [8]:
annotation = pandas.read_csv(annotation_path)
annotation

,Name,Bbox,Class
0,1001958.jpg,"0.7075520833333333,0.5319444444444444,0.282812...",1
1,1001958.jpg,"0.09505208333333333,0.6305555555555555,0.19010...",0
2,1001958.jpg,"0.031510416666666666,0.7434027777777777,0.0630...",0
3,1002155.jpg,"0.8135416666666667,0.6976851851851852,0.371875...",0
4,1002155.jpg,"0.3221354166666667,0.7939814814814815,0.477604...",0
...,...,...,...
1980,1997546.jpg,"0.34661458333333334,0.490625,0.6015625,0.48958...",1
1981,1997602.jpg,"0.7317708333333334,0.25601851851851853,0.30625...",0
1982,1999067.jpg,"0.5630208333333333,0.5020833333333333,0.519791...",1
1983,1999067.jpg,"0.9572916666666667,0.5881944444444445,0.085416...",0


In [9]:
dataset = DetectorDataset(images_path, annotation)

## Model

In [10]:
detection_model = pw_detection.MegaDetectorV6(device=device, pretrained=True)

Ultralytics 8.3.28 🚀 Python-3.10.13 torch-2.2.1 CUDA:0 (Tesla V100-PCIE-32GB, 32494MiB)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs


In [11]:
with open(data_path / "detection_results.dat", 'rb') as file:
    results = pickle.load(file)

In [12]:
def baseline_classify(annotations_pred, confidence_treshold=0.86):
    annotations_pred = annotations_pred.copy()
    annotations_pred["Class"] = (annotations_pred["label"] == 0) & (annotations_pred["confidence"] > confidence_treshold)

    return annotations_pred

In [13]:
annotations_pred = baseline_classify(detection_results_to_annotations(results))

In [14]:
annotations_pred

,Name,Bbox,xyxy_normalized_coords,label,confidence,Class
0,1001958-checkpoint.jpg,"0.7119077682495117, 0.5377415974934896, 0.2591...","[0.5823110580444336, 0.38854726155598956, 0.84...",0,0.921416,True
1,1001958-checkpoint.jpg,"0.09196978211402894, 0.6256720225016277, 0.183...","[5.556344985961914e-05, 0.4938507080078125, 0....",0,0.911797,True
2,1001958-checkpoint.jpg,"0.35349483489990235, 0.8627684275309244, 0.246...","[0.23017826080322265, 0.7784647623697917, 0.47...",0,0.762183,False
3,1001958-checkpoint.jpg,"0.027079170942306517, 0.7385032653808594, 0.05...","[0.0, 0.6669570922851562, 0.054158341884613034...",0,0.598208,False
4,1001958.jpg,"0.7119077682495117, 0.5377415974934896, 0.2591...","[0.5823110580444336, 0.38854726155598956, 0.84...",0,0.921416,True
...,...,...,...,...,...,...
1980,1997546.jpg,"0.3543984889984131, 0.49052821265326607, 0.590...","[0.05928945541381836, 0.2532713572184245, 0.64...",0,0.951897,True
1981,1997602.jpg,"0.7356247901916504, 0.25372191535101996, 0.284...","[0.5932760238647461, 0.00031017727322048613, 0...",0,0.765240,False
1982,1999067.jpg,"0.5605239550272624, 0.5130659951104057, 0.4935...","[0.31377201080322265, 0.2592296600341797, 0.80...",0,0.952601,True
1983,1999067.jpg,"0.9620908737182617, 0.5899592929416233, 0.0755...","[0.924293327331543, 0.3298563639322917, 0.9998...",0,0.855032,False


In [15]:
from misc.metrics import *

In [16]:
metric_value, metric_data = calculate_metric(annotation, annotations_pred)

In [17]:
metric_value

0.43541560859996614

In [18]:
(~metric_data["correct_Bbox"] & metric_data["Class_true"]).sum()

8